Let's start with your project: 

Are you a data scientist? 

I think you are an awesome a data scientist.

### **Problem** 
**Our goal is to create a predictive model that can answer the following question:**

**What kind of people had a better chance of surviving?**

**Data about passengers:**
*   Name
*   Age
*   Gender


## Install and Import Libraries
Let's install PySpark:

## Build Spark Session

In [3]:
import findspark
findspark.init()

from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("practical").getOrCreate()

## Data Loading


You have two datasets: 
* Train  
* Test.

Read two datasets: 
* Train
* Test.



In [12]:
train_Path = "train.csv"
test_Path = "test.csv"

trainDF = spark.read.format("csv")\
                .option("header", "true")\
                .option("inferSchema", "true").load(train_Path)

testDF = spark.read.format("csv")\
                .option("header", "true")\
                .option("inferSchema", "true").load(test_Path)

Let's work with train dataset:

**Confirm if this is a dataframe or not:**

In [13]:
print(trainDF)

DataFrame[PassengerId: int, Survived: int, Pclass: int, Name: string, Sex: string, Age: double, SibSp: int, Parch: int, Ticket: string, Fare: double, Cabin: string, Embarked: string]


**Show 5 rows.**

In [14]:
trainDF.show(5)

+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|PassengerId|Survived|Pclass|                Name|   Sex| Age|SibSp|Parch|          Ticket|   Fare|Cabin|Embarked|
+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|          1|       0|     3|Braund, Mr. Owen ...|  male|22.0|    1|    0|       A/5 21171|   7.25| null|       S|
|          2|       1|     1|Cumings, Mrs. Joh...|female|38.0|    1|    0|        PC 17599|71.2833|  C85|       C|
|          3|       1|     3|Heikkinen, Miss. ...|female|26.0|    0|    0|STON/O2. 3101282|  7.925| null|       S|
|          4|       1|     1|Futrelle, Mrs. Ja...|female|35.0|    1|    0|          113803|   53.1| C123|       S|
|          5|       0|     3|Allen, Mr. Willia...|  male|35.0|    0|    0|          373450|   8.05| null|       S|
+-----------+--------+------+--------------------+------+----+-----+-----+------

**Display schema for the dataset:**

In [15]:
trainDF.printSchema()

root
 |-- PassengerId: integer (nullable = true)
 |-- Survived: integer (nullable = true)
 |-- Pclass: integer (nullable = true)
 |-- Name: string (nullable = true)
 |-- Sex: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- SibSp: integer (nullable = true)
 |-- Parch: integer (nullable = true)
 |-- Ticket: string (nullable = true)
 |-- Fare: double (nullable = true)
 |-- Cabin: string (nullable = true)
 |-- Embarked: string (nullable = true)



**Statistical summary:**

In [17]:
trainDF.describe().show()

+-------+-----------------+-------------------+------------------+--------------------+------+------------------+------------------+-------------------+------------------+-----------------+-----+--------+
|summary|      PassengerId|           Survived|            Pclass|                Name|   Sex|               Age|             SibSp|              Parch|            Ticket|             Fare|Cabin|Embarked|
+-------+-----------------+-------------------+------------------+--------------------+------+------------------+------------------+-------------------+------------------+-----------------+-----+--------+
|  count|              891|                891|               891|                 891|   891|               714|               891|                891|               891|              891|  204|     889|
|   mean|            446.0| 0.3838383838383838| 2.308641975308642|                null|  null| 29.69911764705882|0.5230078563411896|0.38159371492704824|260318.54916792738| 32.20420

In [18]:
trainDF.summary().show()

+-------+-----------------+-------------------+------------------+--------------------+------+------------------+------------------+-------------------+------------------+-----------------+-----+--------+
|summary|      PassengerId|           Survived|            Pclass|                Name|   Sex|               Age|             SibSp|              Parch|            Ticket|             Fare|Cabin|Embarked|
+-------+-----------------+-------------------+------------------+--------------------+------+------------------+------------------+-------------------+------------------+-----------------+-----+--------+
|  count|              891|                891|               891|                 891|   891|               714|               891|                891|               891|              891|  204|     889|
|   mean|            446.0| 0.3838383838383838| 2.308641975308642|                null|  null| 29.69911764705882|0.5230078563411896|0.38159371492704824|260318.54916792738| 32.20420

## EDA - Exploratory Data Analysis

**Display count for the train dataset:**

In [19]:
trainDF.count()

891

**Can you answer this question:** 

**How many people survived, and how many didn't survive?** 

**Please save data in a variable.**

In [51]:
from pyspark.sql.functions import col, expr

survived_not = trainDF.groupby("Survived").count()

**Display your result:**

In [35]:
survived_not.show()

+--------+-----+
|Survived|count|
+--------+-----+
|       1|  342|
|       0|  549|
+--------+-----+



**Can you display your answer in ratio form?(Hint: Use "UDF" Function. (Hint: Use "UDF" Function. This is a hint you can use any method.)**






In [91]:
totalCount = trainDF.count()
survived_not = survived_not.withColumn("Ratios", expr("count")/totalCount)
survived_not.show()

+--------+-----+------------------+
|Survived|count|            Ratios|
+--------+-----+------------------+
|       1|  342|0.3838383838383838|
|       0|  549|0.6161616161616161|
+--------+-----+------------------+



**Can you get the number of males and females?**


In [59]:
trainDF.groupby("Sex").count().show()

+------+-----+
|   Sex|count|
+------+-----+
|female|  314|
|  male|  577|
+------+-----+



**1. What is the average number of survivors of each gender?**

**2. What is the number of survivors of each gender?**

(Hint: Group by the "sex" column. This is a hint you can use any method.)

In [63]:
survivals = trainDF.groupby("Sex", "Survived").count()
survivals.show()

+------+--------+-----+
|   Sex|Survived|count|
+------+--------+-----+
|  male|       0|  468|
|female|       1|  233|
|female|       0|   81|
|  male|       1|  109|
+------+--------+-----+



In [90]:
totalCount = trainDF.count()
survivals = survivals.withColumn("Ratios", expr("count")/totalCount)
survivals.show()

+------+--------+-----+-------------------+
|   Sex|Survived|count|             Ratios|
+------+--------+-----+-------------------+
|  male|       0|  468| 0.5252525252525253|
|female|       1|  233| 0.2615039281705948|
|female|       0|   81|0.09090909090909091|
|  male|       1|  109|  0.122334455667789|
+------+--------+-----+-------------------+



**Create temporary view PySpark:**

In [93]:
trainDF.createOrReplaceTempView("sqlDF")

**How many people survived, and how many didn't survive? By SQL:**

In [96]:
spark.sql(""" SELECT Survived, count(Survived) as counts 
                FROM sqlDF
                GROUP BY Survived""").show()

+--------+------+
|Survived|counts|
+--------+------+
|       1|   342|
|       0|   549|
+--------+------+



**Can you display the number of survivors from each gender as a ratio?**

(Hint: Group by "sex" column. This is a hint you can use any method.)

**Can you do this via SQL?**

In [97]:
spark.sql(""" SELECT Sex, Survived, count(Survived) as counts 
                FROM sqlDF
                GROUP BY Sex, Survived""").show()

+------+--------+------+
|   Sex|Survived|counts|
+------+--------+------+
|  male|       0|   468|
|female|       1|   233|
|female|       0|    81|
|  male|       1|   109|
+------+--------+------+



**Display a ratio for "p-class": SUM(Survived)/count for p-class**


In [100]:
survivesd = trainDF.filter(expr("survived = 1")).count()
p_class = trainDF.groupby("Pclass").count()
p_class = p_class.withColumn("ratios", expr("Pclass")/survivesd)
p_class.show()

+------+-----+--------------------+
|Pclass|count|              ratios|
+------+-----+--------------------+
|     1|  216|0.002923976608187...|
|     3|  491|0.008771929824561403|
|     2|  184|0.005847953216374269|
+------+-----+--------------------+



**Let's take a break and continue after this.**

## Data Cleaning

**First and foremost, we must merge both the train and test datasets. (Hint: The union function can do this.)**



In [268]:
unionDF = trainDF.union(testDF)

**Display count:**

In [269]:
unionDF.count()

1329

**Can you define the number of null values in each column?**


In [270]:
from pyspark.sql.functions import col,isnan, when, count
nullDF = unionDF.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in unionDF.columns])
nullDF.show()

+-----------+--------+------+----+---+---+-----+-----+------+----+-----+--------+
|PassengerId|Survived|Pclass|Name|Sex|Age|SibSp|Parch|Ticket|Fare|Cabin|Embarked|
+-----------+--------+------+----+---+---+-----+-----+------+----+-----+--------+
|          0|       0|     0|   0|  0|265|    0|    0|     0|   0| 1021|       3|
+-----------+--------+------+----+---+---+-----+-----+------+----+-----+--------+



**Create Dataframe for null values**

1. Column
2. Number of missing values.

In [271]:
nullDF = nullDF.select([c for c in nullDF.columns if nullDF.select(c).collect()[0][0]])
nullDF.show()

+---+-----+--------+
|Age|Cabin|Embarked|
+---+-----+--------+
|265| 1021|       3|
+---+-----+--------+



## Preprocessing 

**Create Temporary view PySpark:**

In [272]:
unionDF.createOrReplaceTempView("unionSQL")

**Can you show the "name" column from your temporary table?**

In [273]:
spark.sql(""" SELECT Name FROM unionSQL""").show(5)

+--------------------+
|                Name|
+--------------------+
|Braund, Mr. Owen ...|
|Cumings, Mrs. Joh...|
|Heikkinen, Miss. ...|
|Futrelle, Mrs. Ja...|
|Allen, Mr. Willia...|
+--------------------+
only showing top 5 rows



**Run this code:**

In [274]:
import pyspark.sql.functions as F
unionDF = unionDF.withColumn('Title',F.regexp_extract(F.col("Name"),"([A-Za-z]+)\.",1))
unionDF.createOrReplaceTempView('combinedSQL')

**Display "Title" column and count "Title" column:**

In [275]:
unionDF.groupby("Title").count().show()

+--------+-----+
|   Title|count|
+--------+-----+
|     Don|    1|
|    Miss|  257|
|Countess|    2|
|     Col|    4|
|     Rev|    9|
|    Lady|    2|
|  Master|   56|
|     Mme|    1|
|    Capt|    2|
|      Mr|  786|
|      Dr|   11|
|     Mrs|  186|
|     Sir|    2|
|Jonkheer|    2|
|    Mlle|    4|
|   Major|    3|
|      Ms|    1|
+--------+-----+



**We can see that Dr, Rev, Major, Col, Mlle, Capt, Don, Jonkheer, Countess, Ms, Sir, Lady, and Mme are really rare titles, so create Dictionary and set the value to "rare".**

In [276]:
titles_map = {
    "Dr":'rare', 
    "Rev":'rare', 
    "Major":'rare', 
    "Col":'rare', 
    "Mlle":'rare', 
    "Capt":'rare', 
    "Don":'rare', 
    "Jonkheer":'rare', 
    "Countess":'rare', 
    "Ms":'rare', 
    "Sir":'rare', 
    "Lady":'rare',
    "Mme":'rare',
    "Miss" : "Miss",
    "Master" : "Master",
    "Mr" : "Mr",
    "Mrs" : "Mrs"
}

**Run the function:**

In [277]:
def impute_title(title):
    return titles_map[title]# Title_map is your dictionary. please change this name with your dictionary name.

**Apply the function on "Title" column using UDF:**

In [278]:
from pyspark.sql.functions import udf

convertUDF = udf(lambda x: impute_title(x))

unionDF = unionDF.withColumn("Title", convertUDF(col("Title")))
unionDF.show()

+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+------+
|PassengerId|Survived|Pclass|                Name|   Sex| Age|SibSp|Parch|          Ticket|   Fare|Cabin|Embarked| Title|
+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+------+
|          1|       0|     3|Braund, Mr. Owen ...|  male|22.0|    1|    0|       A/5 21171|   7.25| null|       S|    Mr|
|          2|       1|     1|Cumings, Mrs. Joh...|female|38.0|    1|    0|        PC 17599|71.2833|  C85|       C|   Mrs|
|          3|       1|     3|Heikkinen, Miss. ...|female|26.0|    0|    0|STON/O2. 3101282|  7.925| null|       S|  Miss|
|          4|       1|     1|Futrelle, Mrs. Ja...|female|35.0|    1|    0|          113803|   53.1| C123|       S|   Mrs|
|          5|       0|     3|Allen, Mr. Willia...|  male|35.0|    0|    0|          373450|   8.05| null|       S|    Mr|
|          6|       0|  

**Display "Title" from table and group by "Title" column:**

In [279]:
unionDF.groupby("Title").count().show()

+------+-----+
| Title|count|
+------+-----+
|  rare|   44|
|  Miss|  257|
|Master|   56|
|    Mr|  786|
|   Mrs|  186|
+------+-----+



## **Preprocessing Age**

**Based on the "age" column mean, you will fill in the missing age values:**

**Fill missing with "age" mean:**

In [280]:
mean = unionDF.selectExpr("mean(Age)").collect()[0][0]

unionDF = unionDF.na.fill(mean, subset=["Age"])
unionDF.show()

+-----------+--------+------+--------------------+------+------------------+-----+-----+----------------+-------+-----+--------+------+
|PassengerId|Survived|Pclass|                Name|   Sex|               Age|SibSp|Parch|          Ticket|   Fare|Cabin|Embarked| Title|
+-----------+--------+------+--------------------+------+------------------+-----+-----+----------------+-------+-----+--------+------+
|          1|       0|     3|Braund, Mr. Owen ...|  male|              22.0|    1|    0|       A/5 21171|   7.25| null|       S|    Mr|
|          2|       1|     1|Cumings, Mrs. Joh...|female|              38.0|    1|    0|        PC 17599|71.2833|  C85|       C|   Mrs|
|          3|       1|     3|Heikkinen, Miss. ...|female|              26.0|    0|    0|STON/O2. 3101282|  7.925| null|       S|  Miss|
|          4|       1|     1|Futrelle, Mrs. Ja...|female|              35.0|    1|    0|          113803|   53.1| C123|       S|   Mrs|
|          5|       0|     3|Allen, Mr. Willia..

## **Preprocessing Embarked**

**Select "Embarked" column, count them, order by count Desc, and save in grouped_Embarked variable:**




In [281]:
groupped_Embarked = unionDF.groupby("Embarked").count().orderBy('count', ascending= False)

**Show "groupped_Embarked" your variable:**

In [282]:
groupped_Embarked.show()

+--------+-----+
|Embarked|count|
+--------+-----+
|       S|  962|
|       C|  253|
|       Q|  111|
|    null|    3|
+--------+-----+



**Get max of groupped_Embarked:** 

In [283]:
from pyspark.sql.functions import max
max_val = groupped_Embarked.select(max('count')).collect()[0][0]
groupped_Embarked.select("Embarked").filter(expr("count")==max_val).collect()[0][0]

'S'

**Fill missing values with max 'S' of grouped_Embarked:**

In [284]:
unionDF = unionDF.na.fill(max_val, subset=["Embarked"])
unionDF.show()

+-----------+--------+------+--------------------+------+------------------+-----+-----+----------------+-------+-----+--------+------+
|PassengerId|Survived|Pclass|                Name|   Sex|               Age|SibSp|Parch|          Ticket|   Fare|Cabin|Embarked| Title|
+-----------+--------+------+--------------------+------+------------------+-----+-----+----------------+-------+-----+--------+------+
|          1|       0|     3|Braund, Mr. Owen ...|  male|              22.0|    1|    0|       A/5 21171|   7.25| null|       S|    Mr|
|          2|       1|     1|Cumings, Mrs. Joh...|female|              38.0|    1|    0|        PC 17599|71.2833|  C85|       C|   Mrs|
|          3|       1|     3|Heikkinen, Miss. ...|female|              26.0|    0|    0|STON/O2. 3101282|  7.925| null|       S|  Miss|
|          4|       1|     1|Futrelle, Mrs. Ja...|female|              35.0|    1|    0|          113803|   53.1| C123|       S|   Mrs|
|          5|       0|     3|Allen, Mr. Willia..

## **Preprocessing Cabin**

**Replace "cabin" column with first char from the string:**



In [287]:
def replace(x):
    if x != None:
        return x[0]

    return x

replaceUDF = udf(lambda x: replace(x))
unionDF = unionDF.withColumn("cabin", replaceUDF(col("cabin")))

# unionDF = unionDF.withColumn('Title',F.regexp_extract(F.col("Name"),"([A-Za-z]+)\.",1))

**Show the result:**

In [288]:
unionDF.show()

+-----------+--------+------+--------------------+------+------------------+-----+-----+----------------+-------+-----+--------+------+
|PassengerId|Survived|Pclass|                Name|   Sex|               Age|SibSp|Parch|          Ticket|   Fare|cabin|Embarked| Title|
+-----------+--------+------+--------------------+------+------------------+-----+-----+----------------+-------+-----+--------+------+
|          1|       0|     3|Braund, Mr. Owen ...|  male|              22.0|    1|    0|       A/5 21171|   7.25| null|       S|    Mr|
|          2|       1|     1|Cumings, Mrs. Joh...|female|              38.0|    1|    0|        PC 17599|71.2833|    C|       C|   Mrs|
|          3|       1|     3|Heikkinen, Miss. ...|female|              26.0|    0|    0|STON/O2. 3101282|  7.925| null|       S|  Miss|
|          4|       1|     1|Futrelle, Mrs. Ja...|female|              35.0|    1|    0|          113803|   53.1|    C|       S|   Mrs|
|          5|       0|     3|Allen, Mr. Willia..

**Create the temporary view:**

In [289]:
unionDF.createOrReplaceTempView("finalSQL")

**Select "Cabin" column, count "Cabin" column, Group by "Cabin" column, Order By count DESC**  

In [291]:
unionDF.groupby("cabin").count().orderBy("count", ascending=False).show()

+-----+-----+
|cabin|count|
+-----+-----+
| null| 1021|
|    C|   82|
|    B|   77|
|    D|   52|
|    E|   51|
|    A|   23|
|    F|   18|
|    G|    4|
|    T|    1|
+-----+-----+



**Fill missing values with "U":**

In [292]:
unionDF = unionDF.na.fill("U", subset=["cabin"])
unionDF.show()

+-----------+--------+------+--------------------+------+------------------+-----+-----+----------------+-------+-----+--------+------+
|PassengerId|Survived|Pclass|                Name|   Sex|               Age|SibSp|Parch|          Ticket|   Fare|cabin|Embarked| Title|
+-----------+--------+------+--------------------+------+------------------+-----+-----+----------------+-------+-----+--------+------+
|          1|       0|     3|Braund, Mr. Owen ...|  male|              22.0|    1|    0|       A/5 21171|   7.25|    U|       S|    Mr|
|          2|       1|     1|Cumings, Mrs. Joh...|female|              38.0|    1|    0|        PC 17599|71.2833|    C|       C|   Mrs|
|          3|       1|     3|Heikkinen, Miss. ...|female|              26.0|    0|    0|STON/O2. 3101282|  7.925|    U|       S|  Miss|
|          4|       1|     1|Futrelle, Mrs. Ja...|female|              35.0|    1|    0|          113803|   53.1|    C|       S|   Mrs|
|          5|       0|     3|Allen, Mr. Willia..

**StringIndexer: A label indexer that maps a string column of labels to an ML column of label indices. If the input column is numeric, we cast it to string and index the string values. The indices are in [0, numLabels). By default, this is ordered by label frequencies so the most frequent label gets index 0. The ordering behavior is controlled by setting stringOrderType. Its default value is ‘frequencyDesc’.**

**StringIndexer(inputCol=None, outputCol=None)**

In [293]:
from pyspark.ml.feature import StringIndexer

In [297]:
categoricalCols = [field for (field, dataType) in unionDF.dtypes
                   if dataType == "string"]
indexOutputCols = [x + "_Index" for x in categoricalCols]

stringIndexer = StringIndexer(inputCols=categoricalCols,
                             outputCols=indexOutputCols,
                             handleInvalid='skip')

**OneHotEncoder(inputCols=None, outputCols=None)**

A one-hot encoder that maps a column of category indices to a column of binary vectors, with at most a single one-value per row that indicates the input category index. For example with 5 categories, an input value of 2.0 would map to an output vector of [0.0, 0.0, 1.0, 0.0]. The last category is not included by default (configurable via dropLast), because it makes the vector entries sum up to one, and hence linearly dependent. So an input value of 4.0 maps to [0.0, 0.0, 0.0, 0.0].

In [298]:
from pyspark.ml.feature import OneHotEncoder

oheOutputCols = [x + "_OHE" for x in categoricalCols]
oheEncoder = OneHotEncoder(inputCols=indexOutputCols,
                          outputCols=oheOutputCols)

**VectorAssembler: VectorAssembler(*, inputCols=None, outputCol=None). A feature transformer that merges multiple columns into a vector column.**



In [300]:
numericCols = [field for (field,dataType) in unionDF.dtypes
              if ((dataType=='double')& (field!='price'))]

In [301]:
assemblerInputs = oheOutputCols + numericCols

In [302]:
from pyspark.ml.feature import VectorAssembler
vecAssembler = VectorAssembler(inputCols=assemblerInputs,outputCol='features')

**Use randomSplit function and split data to x_train, and X_test with 80% and 20% Consecutive**

In [307]:
seed = 42
splits = unionDF.randomSplit([0.8, 0.2], seed)
trainDF = splits[0]
testDF = splits[1]

**Pipeline: ML Pipelines provide a uniform set of high-level APIs built on top of DataFrames that help users create and tune practical machine learning pipelines.**

**Build RandomForestClassifier model and use pipeline to fit and transform then display "prediction, Survived, features" columns**

In [306]:
from pyspark.ml.classification import RandomForestClassifier

In [309]:
rf = RandomForestClassifier(featuresCol="features", labelCol='Survived')

stages = [stringIndexer, oheEncoder, vecAssembler, rf]

from pyspark.ml import Pipeline
pipeline = Pipeline(stages=stages)

pipelineModel = pipeline.fit(trainDF) # This line should error


predDF = pipelineModel.transform(testDF)



**Use MulticlassClassificationEvaluator and set the "labelCol" to "Survived",  "predictionCol" to "prediction", "metricName" to "accuracy"** 

In [310]:
# Using RMSE
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [311]:
Evaluator = MulticlassClassificationEvaluator(predictionCol='prediction',
                                         labelCol='Survived',
                                         metricName='accuracy')

acc = Evaluator.evaluate(predDF)
print(f"accurcy = {acc}")

accurcy = 0.7762237762237763


**When you are finished send the project via Google classroom**
**Please let me know if you have any questions.**
* nabieh.mostafa@yahoo.com
* +201015197566 (Whatsapp)

**Don't Hate me, I push you to learn**

**I will help you to become an awesome data engineer.**

**Why did I say that "Data Engineer"?**

**Tricky question, but an optional question, if you would like to know the answer, ask me.**
